In [ ]:
pip install geopy requests


In [ ]:
pip install gradio

In [2]:
import uuid
import gradio as gr
import requests
import heapq
import time
from geopy.distance import geodesic
from threading import Thread

# Coordenadas de la empresa de reparto localizada en Calle Ercilla de Bilbao
empresa_coords = (43.263285, -2.934337)  # Latitud y longitud

# Lista para almacenar pedidos y cola de prioridad
pedidos = []
pedidos_prioritarios = []

# Función para generar ID único
def generar_id_pedido():
    return str(uuid.uuid4())[:8]  # Usa los primeros 8 caracteres de un UUID

# Función para obtener coordenadas de una dirección usando la API de Nominatim
def obtener_coordenadas(direccion):
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": direccion,
        "format": "json",
        "addressdetails": 1
    }
    headers = {
        "User-Agent": "YoTeLoLlevo/1.0 (tuemail@dominio.com)"
    }
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200 and response.json():
        data = response.json()[0]
        return float(data["lat"]), float(data["lon"])
    return None

# Función para calcular el tiempo de preparación de un pedido dependiendo de la cantidad de los paquetes
def calcular_tiempo_preparacion(pedido):
    tiempos = [pedido["Pequeños"]* 2, pedido["Medianos"] * 3, pedido["Grandes"] * 4]
    return max(tiempos) * 60  # Convertir a segundos

# Función para validar la dirección y procesar el pedido
def procesar_pedido(calle, numero, piso, localidad, paquete_pequeño, paquete_mediano, paquete_grande):
    direccion_simple = f"{calle}, {numero} {localidad.capitalize()}"
    coords = obtener_coordenadas(direccion_simple)
    if not coords:
        return f"Error: No se pudo determinar la ubicación de la dirección: {direccion_simple}"

    distancia = geodesic(empresa_coords, coords).km
    if distancia > 10:
        return f"Error: La dirección está fuera de nuestro radio de entrega (Distancia: {distancia:.2f} km)"

    if not (paquete_pequeño or paquete_mediano or paquete_grande):
        return "Por favor, selecciona al menos un paquete antes de finalizar el pedido."

    direccion_completa = f"{calle}, {numero}, Piso {piso}, {localidad.capitalize()}"
    pedido_id = generar_id_pedido()

    # Calcular unidades ocupadas en el vehículo de reparto
    unidades = paquete_pequeño * 0.5 + paquete_mediano * 0.75 + paquete_grande * 1

    pedido = {
        "ID": pedido_id,
        "Dirección": direccion_completa,
        "Pequeños": paquete_pequeño,
        "Medianos": paquete_mediano,
        "Grandes": paquete_grande,
        "Unidades": unidades,
        "Estado": "En espera",
        "Tiempo restante": "0:00"
    }
    pedidos.append(pedido)

    heapq.heappush(pedidos_prioritarios, (len(pedidos), pedido))

    return f"Pedido recibido con ID: {pedido_id}\nPaquetes - Pequeños: {paquete_pequeño}, Medianos: {paquete_mediano}, Grandes: {paquete_grande}\nDirección: {direccion_completa}\nUnidades: {unidades}\nEstado: En espera"

# Función para gestionar la preparación de los paquetes
def gestionar_preparacion():
    while True:
        if not pedidos_prioritarios:
            time.sleep(1)  # Esperar antes de verificar nuevamente
            continue

        vehiculo_actual = []  # Paquetes actualmente en preparación
        tiempo_preparacion = 0  # Tiempo total de preparación en segundos

        # Cargar el vehículo respetando la capacidad
        while pedidos_prioritarios:
            _, pedido = heapq.heappop(pedidos_prioritarios)

            if sum(p[1]["Unidades"] for p in vehiculo_actual) + pedido["Unidades"] <= 10:
                vehiculo_actual.append((calcular_tiempo_preparacion(pedido), pedido))
                # Actualizar estado a "En preparación" en la lista global de pedidos
                for p in pedidos:
                    if p["ID"] == pedido["ID"]:
                        p["Estado"] = "En preparación"
                        minutos, segundos = divmod(calcular_tiempo_preparacion(pedido), 60)
                        p["Tiempo restante"] = f"{minutos}:{segundos:02d}"
            else:
                # Reinsertar el pedido si no cabe
                heapq.heappush(pedidos_prioritarios, (len(pedidos), pedido))
                break

        # Calcular el tiempo de preparación del lote basado en el pedido más largo
        if vehiculo_actual:
            tiempo_preparacion = max(t[0] for t in vehiculo_actual)

        # Actualizar la barra de progreso
        for t in range(tiempo_preparacion, 0, -1):
            for _, pedido in vehiculo_actual:
                for p in pedidos:
                    if p["ID"] == pedido["ID"]:
                        minutos, segundos = divmod(t, 60)
                        p["Tiempo restante"] = f"{minutos}:{segundos:02d}"
            time.sleep(1)

        # Actualizar estados tras la preparación
        for _, pedido in vehiculo_actual:
            for p in pedidos:
                if p["ID"] == pedido["ID"]:
                    p["Estado"] = "Listo para entrega"
                    p["Tiempo restante"] = "0:00"

# Iniciar el hilo de preparación
Thread(target=gestionar_preparacion, daemon=True).start()

# Función para mostrar la lista de pedidos con estados
def mostrar_pedidos():
    if not pedidos:
        return "No hay pedidos registrados."

    listado = "\n".join([f"ID: {p['ID']} | Dirección: {p['Dirección']} | Pequeños: {p['Pequeños']} | Medianos: {p['Medianos']} | Grandes: {p['Grandes']} | Unidades: {p['Unidades']:.2f} | Estado: {p['Estado']} | Tiempo restante: {p['Tiempo restante']}" for p in pedidos])
    return listado

# Interfaz de Gradio
with gr.Blocks(theme=gr.themes.Soft()) as app:  # Usamos un tema oscuro
    gr.Markdown("# Sistema de Pedidos - YoTeLoLlevo")

    with gr.Tabs():
        # Pestaña para Pedidos (combinando "Realizar Pedido" y "Gestión de Pedidos")
        with gr.Tab("Pedidos"):
            with gr.Row():  # Usamos Row para colocar "Realizar Pedido" y "Gestión de Pedidos" uno al lado del otro
                with gr.Column():  # Columna para "Realizar Pedido"
                    gr.Markdown("## Realizar Pedido") 
                    #salida = gr.Textbox(label="Estado del Pedido", interactive=False)
                    # Agrupamos Calle y Localidad en la misma fila
                    with gr.Row():
                        calle_input = gr.Textbox(label="Introduce tu calle", placeholder="Ejemplo: Calle Ercilla")
                        localidad_input = gr.Textbox(label="Introduce tu localidad", placeholder="Ejemplo: Bilbao")
                    
                    # Agrupamos Número y Piso en la misma fila
                    with gr.Row():
                        numero_input = gr.Textbox(label="Número", placeholder="Ejemplo: 22")
                        piso_input = gr.Textbox(label="Piso", placeholder="Ejemplo: 3D")
                        
                    # Agrupamos las cantidades de paquetes en la misma fila
                    with gr.Row():
                        paquete_pequeño_input = gr.Number(label="Cantidad de Paquetes Pequeños", value=0, precision=0)
                        paquete_mediano_input = gr.Number(label="Cantidad de Paquetes Medianos", value=0, precision=0)
                        paquete_grande_input = gr.Number(label="Cantidad de Paquetes Grandes", value=0, precision=0)

                    finalizar_btn = gr.Button("Finalizar pedido")
                    #salida = gr.Textbox(label="Estado del Pedido", interactive=False)

                    
                with gr.Column():  # Columna para "Gestión de Pedidos"
                    gr.Markdown("## Gestión de Pedidos")
                    # Agrupamos el Estado del Pedido en la misma fila que el título
                    with gr.Row():
                        salida = gr.Textbox(label="Estado del Pedido", interactive=False)
                    listado_pedidos = gr.Textbox(label="Listado de Pedidos", interactive=False, lines=10)
                    actualizar_btn = gr.Button("Actualizar listado")

                    actualizar_btn.click(mostrar_pedidos, inputs=[], outputs=listado_pedidos)

            finalizar_btn.click(procesar_pedido, inputs=[calle_input, numero_input, piso_input, localidad_input, paquete_pequeño_input, paquete_mediano_input, paquete_grande_input], outputs=salida)
                
# Ejecutar la aplicación
app.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Calle Ercilla, 22 Bilbao
(43.2635245, -2.9341272)
Calle Ercilla, 22 Bilbao
(43.2635245, -2.9341272)
